# Rf 5 0 4_Simwstool
Organisation and simultaneous fits: using RooSimWSTool to construct a simultaneous p.d.f that is built of variations
of an input p.d.f



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:55 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooCategory.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooConstVar.h"
#include "RooPolynomial.h"
#include "RooSimultaneous.h"
#include "RooAddPdf.h"
#include "RooWorkspace.h"
#include "RooSimWSTool.h"
#include "RooPlot.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "TFile.h"
#include "TH1.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Create master pdf
 ---------------------------------

Construct gauss(x,m,s)

In [3]:
RooRealVar x("x", "x", -10, 10);
RooRealVar m("m", "m", 0, -10, 10);
RooRealVar s("s", "s", 1, -10, 10);
RooGaussian gauss("g", "g", x, m, s);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Construct poly(x,p0)

In [4]:
RooRealVar p0("p0", "p0", 0.01, 0., 1.);
RooPolynomial poly("p", "p", x, p0);

Construct model = f*gauss(x) + (1-f)*poly(x)

In [5]:
RooRealVar f("f", "f", 0.5, 0., 1.);
RooAddPdf model("model", "model", RooArgSet(gauss, poly), f);

Create category observables for splitting
 ----------------------------------------------------------------------------------

Define two categories that can be used for splitting

In [6]:
RooCategory c("c", "c");
c.defineType("run1");
c.defineType("run2");

RooCategory d("d", "d");
d.defineType("foo");
d.defineType("bar");

Setup simwstool
 -----------------------------

Import ingredients in a workspace

In [7]:
RooWorkspace w("w", "w");
w.import(RooArgSet(model, c, d));

[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooAddPdf::model
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooGaussian::g
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::x
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::m
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::s
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::f
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooPolynomial::p
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::p0
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooCategory::c
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooCategory::d


Make sim builder tool

In [8]:
RooSimWSTool sct(w);

Build a simultaneous model with one split
 ---------------------------------------------------------------------------------

Construct a simultaneous p.d.f with the following form

 model_run1(x) = f*gauss_run1(x,m_run1,s) + (1-f)*poly
 model_run2(x) = f*gauss_run2(x,m_run2,s) + (1-f)*poly
 simpdf(x,c) = model_run1(x) if c=="run1"
             = model_run2(x) if c=="run2"

 Returned p.d.f is owned by the workspace

In [9]:
RooSimultaneous *model_sim = sct.build("model_sim", "model", SplitParam("m", "c"));

Splitrule for p.d.f model with state list 
 parameter m is split with constraint in categories (c)
[#1] INFO:ObjectHandling -- RooSimWSTool::executeBuild: list of prototype pdfs (model)
[#1] INFO:ObjectHandling -- RooSimWSTool::executeBuild: list of splitting categories (c)
[#1] INFO:ObjectHandling -- RooSimPdfBuilder::executeBuild: processing prototype pdf model
[#1] INFO:ObjectHandling -- RooSimWSTool::executeBuild: configured customizers for all prototype pdfs
[#1] INFO:ObjectHandling -- RooSimWSTool::executeBuild: Customizing prototype pdf model for mode run1
[#1] INFO:ObjectHandling -- RooSimWSTool::executeBuild: Customizing prototype pdf model for mode run2
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooSimultaneous::model_sim
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooAddPdf::model_run1
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooGaussian::g_run1
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar:

Print tree structure of model

In [10]:
model_sim->Print("t");

0x7f0af852e7f0 RooSimultaneous::model_sim = 1 [Auto,Dirty] 
  0x7f0af853b2e0/V- RooAddPdf::model_run1 = 1 [Auto,Dirty] 
    0x7f0af859ada0/V- RooGaussian::g_run1 = 1 [Auto,Dirty] 
      0x7f0af859c750/V- RooRealVar::m_run1 = 0
      0x7f0af851c8e0/V- RooRealVar::x = 0
      0x7f0af8470960/V- RooRealVar::s = 1
    0x7f0af8512cb0/V- RooRealVar::f = 0.5
    0x7f0af8504aa0/V- RooPolynomial::p = 1 [Auto,Dirty] 
      0x7f0af851c8e0/V- RooRealVar::x = 0
      0x7f0af8504f70/V- RooRealVar::p0 = 0.01
  0x7f0af859cf30/V- RooAddPdf::model_run2 = 1 [Auto,Dirty] 
    0x7f0af8544e40/V- RooGaussian::g_run2 = 1 [Auto,Dirty] 
      0x7f0af8545e50/V- RooRealVar::m_run2 = 0
      0x7f0af851c8e0/V- RooRealVar::x = 0
      0x7f0af8470960/V- RooRealVar::s = 1
    0x7f0af8512cb0/V- RooRealVar::f = 0.5
    0x7f0af8504aa0/V- RooPolynomial::p = 1 [Auto,Dirty] 
      0x7f0af851c8e0/V- RooRealVar::x = 0
      0x7f0af8504f70/V- RooRealVar::p0 = 0.01
  0x7f0af846b1d0/V- RooCategory::c = run2(idx = 1)



Adjust model_sim parameters in workspace

In [11]:
w.var("m_run1")->setVal(-3);
w.var("m_run2")->setVal(+3);

Print contents of workspace

In [12]:
w.Print("v");


RooWorkspace(w) w contents

variables
---------
(c,d,f,m,m_run1,m_run2,p0,s,x)

p.d.f.s
-------
RooGaussian::g[ x=x mean=m sigma=s ] = 1
RooGaussian::g_run1[ x=x mean=m_run1 sigma=s ] = 0.011109
RooGaussian::g_run2[ x=x mean=m_run2 sigma=s ] = 0.011109
RooAddPdf::model[ f * g + [%] * p ] = 1
RooAddPdf::model_run1[ f * g_run1 + [%] * p ] = 0.505554
RooAddPdf::model_run2[ f * g_run2 + [%] * p ] = 0.505554
RooSimultaneous::model_sim[ indexCat=c run1=model_run1 run2=model_run2 ] = 0.505554
RooPolynomial::p[ x=x coefList=(p0) ] = 1



Build a simultaneous model with product split
 -----------------------------------------------------------------------------------------

Build another simultaneous p.d.f using a composite split in states c x d

In [13]:
RooSimultaneous *model_sim2 = sct.build("model_sim2", "model", SplitParam("p0", "c,d"));

Splitrule for p.d.f model with state list 
 parameter p0 is split with constraint in categories (c,d)
[#1] INFO:ObjectHandling -- RooSimWSTool::executeBuild: list of prototype pdfs (model)
[#1] INFO:ObjectHandling -- RooSimWSTool::executeBuild: list of splitting categories (c,d)
[#1] INFO:ObjectHandling -- RooSimPdfBuilder::executeBuild: processing prototype pdf model
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooMultiCategory::c,d
[#1] INFO:ObjectHandling -- RooSimWSTool::executeBuild: configured customizers for all prototype pdfs
[#1] INFO:ObjectHandling -- RooSimWSTool::executeBuild: Customizing prototype pdf model for mode {run1;foo}
[#1] INFO:ObjectHandling -- RooSimWSTool::executeBuild: Customizing prototype pdf model for mode {run2;foo}
[#1] INFO:ObjectHandling -- RooSimWSTool::executeBuild: Customizing prototype pdf model for mode {run1;bar}
[#1] INFO:ObjectHandling -- RooSimWSTool::executeBuild: Customizing prototype pdf model for mode {run2;bar}
[#1] INFO:O

Print tree structure of this model

In [14]:
model_sim2->Print("t");

0x7f0af8598400 RooSimultaneous::model_sim2 = 1 [Auto,Dirty] 
  0x7f0af858d860/V- RooSuperCategory::model_sim2_index = {run2;bar}(idx = 3)
 [Auto,Clean] 
    0x7f0af846b1d0/VS RooCategory::c = run2(idx = 1)

    0x7f0af8468ae0/VS RooCategory::d = bar(idx = 1)

  0x7f0af857d300/V- RooAddPdf::model_{run1;foo} = 1 [Auto,Dirty] 
    0x7f0af85841d0/V- RooPolynomial::p_{run1;foo} = 1 [Auto,Dirty] 
      0x7f0af8585140/V- RooRealVar::p0_{run1;foo} = 0.01
      0x7f0af851c8e0/V- RooRealVar::x = 0
    0x7f0af8077dc0/V- RooGaussian::g = 1 [Auto,Dirty] 
      0x7f0af851c8e0/V- RooRealVar::x = 0
      0x7f0af8079d40/V- RooRealVar::m = 0
      0x7f0af8470960/V- RooRealVar::s = 1
    0x7f0af8512cb0/V- RooRealVar::f = 0.5
  0x7f0af858b890/V- RooAddPdf::model_{run2;foo} = 1 [Auto,Dirty] 
    0x7f0af858c990/V- RooPolynomial::p_{run2;foo} = 1 [Auto,Dirty] 
      0x7f0af8567040/V- RooRealVar::p0_{run2;foo} = 0.01
      0x7f0af851c8e0/V- RooRealVar::x = 0
    0x7f0af8077dc0/V- RooGaussian::g = 1 [Auto,Dirt